In [3]:
import kagglehub

d:\Personal\fine-tune-model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [5]:
path = kagglehub.dataset_download("thedevastator/2019-e-commerce-product-listings-from-amazon-ind")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\akash\.cache\kagglehub\datasets\thedevastator\2019-e-commerce-product-listings-from-amazon-ind\versions\2


In [6]:
import pandas as pd

data = pd.read_csv(f'{path}/home_sdf_marketing_sample_for_amazon_in-ecommerce__20191001_20191031__30k_data.csv');

print(data.head())

   index                           Uniq Id            Crawl Timestamp  \
0      0  eb49cc038190f6f03c272f79fbbce894  2019-10-30 11:38:11 +0000   
1      1  1657cc30c438affede6a5060d6847363  2019-10-31 15:46:54 +0000   
2      2  41654633cce38c8650690f6dbac01fd3  2019-10-30 09:53:23 +0000   
3      3  08b1bd85c3efc2d7aa556fd79b073382  2019-10-29 16:16:52 +0000   
4      4  3ac3f213732512d1d11bb73ab3b1900f  2019-10-31 09:32:06 +0000   

                  Category                                      Product Title  \
0                Skin Care   Lee posh Lactic Acid 60% Anti ageing Pigmenta...   
1                Skin Care   Branded SLB Works New 1.5mm Titanium 1200 nee...   
2                Skin Care   Generic 1 Pc brand snail eye cream remove dar...   
3                Skin Care   Generic Anti Snoring Snore Stopper Sleep Apne...   
4  Grocery & Gourmet Foods   Harveys Crunchy & Creame Gourmet Delicacies C...   

                                 Product Description      Brand  \
0  PROF

In [7]:
product_details = data.iloc[:,[0,1,3,4,5,6,8,9,13,15]]
product_details = product_details.dropna(subset=['Uniq Id','Product Title','Product Description','Brand','Stock Availibility'])
print(product_details.iloc[0])

index                                                                  0
Uniq Id                                 eb49cc038190f6f03c272f79fbbce894
Category                                                       Skin Care
Product Title           Lee posh Lactic Acid 60% Anti ageing Pigmenta...
Product Description    PROFESSIONAL GRADE Face Peel: this peel stimul...
Brand                                                           Lee Posh
Mrp                                                              2000.00
Price                                                             799.00
Stock Availibility                                                   YES
Image Urls             https://images-na.ssl-images-amazon.com/images...
Name: 0, dtype: object


In [8]:
first_five_prods = product_details.head()

In [9]:
class Product_Listing:
    def __init__(self,index,uniq_id,category,product_title,product_description,brand,mrp,price,stock_avail,image_urls):
        self.index = index;
        self.uniq_id = uniq_id;
        self.category = category;
        self.product_title = product_title
        self.product_description = product_description
        self.brand = brand
        self.mrp = mrp
        self.price = price
        self.stock_avail = stock_avail
        self.image_urls = []
        splitted_image_urls = image_urls.split("|");
        for url in splitted_image_urls:
            self.image_urls.append(url)
            
    def to_dict(self):
        return {
            "index" : self.index,
            "uniq_id": self.uniq_id,
            "category": self.category,
            "product_title": self.product_title,
            "product_description": self.product_description,
            "brand": self.brand,
            "mrp": self.mrp,
            "price": self.price,
            "stock_avail": self.stock_avail,
            "image_urls": self.image_urls
        }

In [10]:
0,1,3,4,5,6,8,9,13,15
product_listings = []

for index,row in product_details.iterrows():
    product = Product_Listing(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9])
    product_listings.append(product.to_dict())
    
print(product_listings[0])

C:\Users\akash\AppData\Local\Temp\ipykernel_14328\442450352.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  product = Product_Listing(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9])


{'index': 0, 'uniq_id': 'eb49cc038190f6f03c272f79fbbce894', 'category': 'Skin Care', 'product_title': ' Lee posh Lactic Acid 60% Anti ageing Pigmentation Removing Glow Peel ', 'product_description': 'PROFESSIONAL GRADE Face Peel: this peel stimulates collagen production, reducing the appearance of wrinkles, fine lines, and hyper pigmentation in the skin by increasing cell regeneration Highly effective professional strength superficial solution Read direction for use on bottle of product & if any query call customer care', 'brand': 'Lee Posh', 'mrp': '2000.00', 'price': '799.00', 'stock_avail': 'YES', 'image_urls': ['https://images-na.ssl-images-amazon.com/images/I/41l0RjF1XIL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41kBVM4KPwL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41kBVM4KPwL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41GL79CxubL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transpar

In [11]:
import json
with open("products_listing.json",'w') as json_file:
    json.dump(product_listings,json_file,indent=4)

In [12]:
%pip install mistralai

Note: you may need to restart the kernel to use updated packages.


In [13]:
import config
from mistralai import Mistral

api_key = config.API_KEY

model = "mistral-large-latest"

client = Mistral(api_key=api_key)

In [14]:
with open('products_listing.json','r') as json_file:
    data = json.load(json_file)

In [15]:
json_listings = data

In [19]:
input_output_listings = []
for index in range(201):
    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "user",
                "content": f"""Create an engaging social media post to promote the following skincare product.Include a catchy introduction, highlight the key benefits, and encourage followers to make apurchase. Use an enthusiastic and inviting tone.

                Product Information:

                Product Title: {json_listings[index]['product_title']}
                Category: {json_listings[index]['category']}
                Description: {json_listings[index]['product_description']}
                Brand: {json_listings[index]['brand']}
                Price: {json_listings[index]['price']} (MRP: {json_listings[index]['mrp']})
                Availability: In Stock
                Make sure to create a sense of urgency and include a call to action!""",
            },
        ]
    )
    input = chat_response.choices[0].message.content
    output = json_listings[index]
    input_output_listings.append({
        "input": input,
        "output": output
        })


with open("listings_input_output.json",'w') as json_file:
        json.dump(input_output_listings,json_file,indent=4)

index': 0, 'uniq_id': 'eb49cc038190f6f03c272f79fbbce894', 'category': 'Skin Care', 'product_title': ' Lee posh Lactic Acid 60% Anti ageing Pigmentation Removing Glow Peel ', 'product_description': 'PROFESSIONAL GRADE Face Peel: this peel stimulates collagen production, reducing the appearance of wrinkles, fine lines, and hyper pigmentation in the skin by increasing cell regeneration Highly effective professional strength superficial solution Read direction for use on bottle of product & if any query call customer care', 'brand': 'Lee Posh', 'mrp': '2000.00', 'price': '799.00', 'stock_avail': 'YES'
